In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from bh24_literature_mining.biotools import get_biotools

# Get biotools from the biotoolspub.tsv
biotools = get_biotools("../biotoolspub/biotoolspub.tsv")
print(len(biotools))

15005


In [22]:
from pathlib import Path
from bh24_literature_mining.europepmc_api import identify_tool_mentions_using_europepmc

tool_occurrences_df = identify_tool_mentions_using_europepmc(biotools[:1])
print(tool_occurrences_df.shape)


(4, 4)


In [23]:
tool_occurrences_df.sample(1)

,PMCID,Sentence,NER_Tags,Topics
2,PMC11286849,The identified proteins also included 52 (70%)...,"[(120, 129, SubtiWiki, subtiwiki)]","Molecular interactions, pathways and networks,..."


In [33]:
# Save the results to a CSV file
tool_occurrences_df.to_csv("../biotoolspub/tmp.csv", index=False)

In [28]:
from bh24_literature_mining.europepmc_api import split_train_test_dev
train_df, test_df, dev_df = split_train_test_dev(tool_occurrences_df)
# Verify the sizes
print("Size of df:", len(tool_occurrences_df))
print("Train set size:", len(train_df))
print("Test set size:", len(test_df))
print("Dev set size:", len(dev_df))
dev_df.sample(n=min(10, len(dev_df)))

Size of df: 4
Train set size: 2
Test set size: 1
Dev set size: 1


,PMCID,Sentence,NER_Tags,Topics
3,PMC11286849,The localization of 52 of these 445 proteins i...,"[(72, 81, SubtiWiki, subtiwiki)]","Molecular interactions, pathways and networks,..."


In [ ]:
import csv

from bh24_literature_mining.europepmc_api import convert_to_IOB_format_from_df


output_folder = '../data/'

# Convert train, dev, and test dataframes to IOB format
convert_to_IOB_format_from_df(train_df, output_folder, 'train_IOB.tsv')
convert_to_IOB_format_from_df(dev_df, output_folder, 'dev_IOB.tsv')
convert_to_IOB_format_from_df(test_df, output_folder, 'test_IOB.tsv')

Processing rows:   0%|          | 0/2 [05:02<?, ?it/s]


TypeError: '<' not supported between instances of 'str' and 'int'